In [1]:
# # Test open PyCharm
# import os
# os.system("wmctrl -a 'PyCharm'")
# os.system("wmctrl -a 'Postman'")

In [2]:
# Generate boss data 
import os 
import random
from shutil import copyfile

for path, subdirs, files in os.walk('./images'):
    if path != './images/toanpv':
        random.shuffle(files)
        for name in files[:4]:
            img_path = os.path.join(path, name)
            copyfile(img_path, './data/other/' + name)

In [3]:
# Build neural network 
import numpy as np
import cv2

IMAGE_SIZE = 224
IMAGE_DATA = './data'

images = []
labels = []

import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from IPython.display import Image
from keras.optimizers import Adam

model = keras.applications.mobilenet.MobileNet(classes=2, weights=None)

def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

# Generate dataset

for path, subdirs, files in os.walk(IMAGE_DATA):
    for name in files:
        img_path = os.path.join(path, name)
        images.append(prepare_image(img_path))
        labels.append(path.split('/')[-1])

Using TensorFlow backend.


In [4]:
images = np.array(images)

In [5]:
images = np.squeeze(images, axis=1)

In [6]:
mapped_labels = list(map(lambda x: 1 if x == 'boss' else 0, labels))

from keras.utils import np_utils

y_data = np_utils.to_categorical(mapped_labels)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, y_data, test_size=0.2)

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 32)      0         
__________

In [9]:
model.compile(loss='categorical_crossentropy',
                           optimizer='adam',
                           metrics=['accuracy'])

In [10]:
from keras.callbacks import ModelCheckpoint

model_file = "boss_model/model_mobilenet.hdf5"

checkpoint = ModelCheckpoint(model_file, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [11]:
model.fit(x=X_train, y=y_train, batch_size=16, epochs=100, verbose=1, validation_data=(X_test, y_test), callbacks=callbacks_list)

Train on 489 samples, validate on 123 samples
Epoch 1/100
489/489 [==============================] - 6s 11ms/step - loss: 0.1362 - acc: 0.9366 - val_loss: 1.6160 - val_acc: 0.3902

Epoch 00001: val_loss improved from inf to 1.61605, saving model to boss_model/model_mobilenet.hdf5
Epoch 2/100
489/489 [==============================] - 3s 6ms/step - loss: 0.0731 - acc: 0.9918 - val_loss: 0.0974 - val_acc: 0.9919

Epoch 00002: val_loss improved from 1.61605 to 0.09741, saving model to boss_model/model_mobilenet.hdf5
Epoch 3/100
489/489 [==============================] - 3s 7ms/step - loss: 0.0465 - acc: 0.9918 - val_loss: 3.2883 - val_acc: 0.0081

Epoch 00003: val_loss did not improve
Epoch 4/100
489/489 [==============================] - 3s 7ms/step - loss: 0.0105 - acc: 0.9980 - val_loss: 10.1264 - val_acc: 0.0081

Epoch 00004: val_loss did not improve
Epoch 5/100
489/489 [==============================] - 3s 7ms/step - loss: 0.0019 - acc: 1.0000 - val_loss: 0.1513 - val_acc: 0.9431

Ep

KeyboardInterrupt: 

In [ ]:
model.save('boss_model/model.h5')

In [ ]:
x = prepare_image('images/toanpv/1546913999.2041926.png')

np.argmax(model.predict(x)[0])